In [136]:
# crawl the website under https://www.studierendenwerk-stuttgart.de/essen/speiseplan

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime
import os
import time
import random
import json
import numpy as np
import matplotlib.pyplot as plt

In [137]:
# POST /inc/ajax-php_konnektor.inc.php HTTP/1.1
# Content-Type: application/x-www-form-urlencoded; charset=UTF-8
# Accept: */*
# Accept-Language: en-us
# Accept-Encoding: gzip, deflate, br
# Host: sws2.maxmanager.xyz
# Origin: https://sws2.maxmanager.xyz
# User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Safari/605.1.15
# Connection: keep-alive
# Referer: https://sws2.maxmanager.xyz/index.php?mode=bed
# Content-Length: 95
# Cookie: domain=sws2.maxmanager.xyz; locId=2; savekennzfilterinput=0; splsws=rhom9pct0fu7q8rvhg6i51m0sc
# X-Requested-With: XMLHttpRequest

# Request Data
# MIME Type: application/x-www-form-urlencoded; charset=UTF-8
# func: make_spl
# locId: 2
# date: 2023-06-29
# lang: de
# startThisWeek: 2023-06-26
# startNextWeek: 2023-07-03


def get_date_or_next_monday():
    if datetime.date.today().weekday() == 5:
        # get the date of the day after tomorrow
        return (datetime.date.today() + datetime.timedelta(days=2)).strftime('%Y-%m-%d')
    # check if today is sunday
    elif datetime.date.today().weekday() == 6:
        # get the date of tomorrow
        return (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    else:
        return datetime.date.today().strftime('%Y-%m-%d')

date = get_date_or_next_monday()

request = requests.post(
    url='https://sws2.maxmanager.xyz/inc/ajax-php_konnektor.inc.php',
    headers={
    #     'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    #     'Accept': '*/*',
    #     'Accept-Language': 'en-us',
    #     'Accept-Encoding': 'gzip, deflate, br',
        'Host': 'sws2.maxmanager.xyz',
        'Origin': 'https://sws2.maxmanager.xyz',
    #     'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Safari/605.1.15',
    #     'Connection': 'keep-alive',
        'Referer': 'https://sws2.maxmanager.xyz/index.php?mode=bed',
    #     'Content-Length': '95',
        'Cookie': 'domain=sws2.maxmanager.xyz; locId=2; savekennzfilterinput=0; splsws=rhom9pct0fu7q8rvhg6i51m0sc',
    #     'X-Requested-With': 'XMLHttpRequest'
    },
    data={
        'func': 'make_spl',
        'locId': '2',
        'date': {date},
        'lang': 'de',
        'startThisWeek': {
            # get the date of monday of the current week
            (datetime.date.today() - datetime.timedelta(days=datetime.date.today().weekday())).strftime('%Y-%m-%d')
        },
        'startNextWeek': {
            # get the date of monday of the next week
            (datetime.date.today() - datetime.timedelta(days=datetime.date.today().weekday()) + datetime.timedelta(days=7)).strftime('%Y-%m-%d')
        }
    }
)

In [138]:
# parse the html with BeautifulSoup
soup = BeautifulSoup(request.text, 'html.parser')

In [139]:
class Essen:
    def __init__(self, name, foto, preis, vegan, co2, nährwerte):
        self.name = name
        self.foto = foto
        self.preis = preis
        self.vegan = vegan
        self.co2 = co2
        self.nährwerte = nährwerte

In [140]:
essens_liste = []

for essen_div in soup.find_all(class_='row splMeal'):
    name = essen_div.find('span').text.strip()
    foto = essen_div.find('img')['src']
    preis = 'leer' #essen_div.find(class_='col-md-2 col-sm-3 visible-sm-block visible-md-block visible-lg-block').div.text.strip()
    vegan_icon = essen_div.find(class_='iconLarge')
    vegan = True if vegan_icon else False
    co2 = essen_div.find(class_='azn hidden size-13').find_all('div')[0].text.strip()
    nährwerte = essen_div.find(class_='azn hidden size-13').find_all('div')[1].text.strip()

    # find the price, it is a div that contains an € sign and a number in the text
    for div in essen_div.find_all('div'):
        if '€' in div.text:
            preis = div.text.strip()
            break

    essen = Essen(name, foto, preis, vegan, co2, nährwerte)
    essens_liste.append(essen)

In [141]:
class EssenDetailed:
    def __init__(self, name, foto, preis, vegan, co2, brennwert, fett, ges_fett, kohlenhydrate, zucker, eiweiß, salz):
        self.name = name
        self.foto = foto
        self.preis = preis
        self.vegan = vegan
        self.co2 = co2
        self.brennwert = brennwert
        self.fett = fett
        self.ges_fett = ges_fett
        self.kohlenhydrate = kohlenhydrate
        self.zucker = zucker
        self.eiweiß = eiweiß
        self.salz = salz

In [142]:
essens_liste_detailled = []

for essen_div in soup.find_all(class_='row splMeal'):
    name = essen_div.find('span').text.strip()
    foto = essen_div.find('img')['src']
    preis = 'leer'#essen_div.find(class_='col-md-2 col-sm-3 visible-sm-block visible-md-block visible-lg-block').div.text.strip()
    vegan_icon = essen_div.find(class_='iconLarge')
    vegan = True if vegan_icon else False
    co2 = essen_div.find(class_='azn hidden size-13').find_all('div')[0].text.strip()
    nährwerte_div = essen_div.find(class_='azn hidden size-13').find_all('div')[1]
    
    brennwert = nährwerte_div.find('span', text='Brennwert:')
    brennwert = brennwert.next_sibling.strip() if brennwert else ''
    
    fett = nährwerte_div.find('span', text='Fett:')
    fett = fett.next_sibling.strip() if fett else ''
    
    ges_fett = nährwerte_div.find('span', text=' - davon ges. FS:')
    ges_fett = ges_fett.next_sibling.strip() if ges_fett else ''
    
    kohlenhydrate = nährwerte_div.find('span', text='Kohlenhydrate:')
    kohlenhydrate = kohlenhydrate.next_sibling.strip() if kohlenhydrate else ''
    
    zucker = nährwerte_div.find('span', text=' - davon Zucker:')
    zucker = zucker.next_sibling.strip() if zucker else ''
    
    eiweiß = nährwerte_div.find('span', text='Eiweiß:')
    eiweiß = eiweiß.next_sibling.strip() if eiweiß else ''
    
    salz = nährwerte_div.find('span', text='Salz:')
    salz = salz.next_sibling.strip() if salz else ''

    # find the price, it is a div that contains an € sign and a number in the text
    for div in essen_div.find_all('div'):
        if '€' in div.text:
            preis = div.text.strip()
            break

    essen = EssenDetailed(name, foto, preis, vegan, co2, brennwert, fett, ges_fett, kohlenhydrate, zucker, eiweiß, salz)
    essens_liste_detailled.append(essen)

/var/folders/m9/14p22wk917lfcwqnfzhgq73c0000gn/T/ipykernel_51172/2389888521.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  brennwert = nährwerte_div.find('span', text='Brennwert:')
/var/folders/m9/14p22wk917lfcwqnfzhgq73c0000gn/T/ipykernel_51172/2389888521.py:15: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  fett = nährwerte_div.find('span', text='Fett:')
/var/folders/m9/14p22wk917lfcwqnfzhgq73c0000gn/T/ipykernel_51172/2389888521.py:18: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  ges_fett = nährwerte_div.find('span', text=' - davon ges. FS:')
/var/folders/m9/14p22wk917lfcwqnfzhgq73c0000gn/T/ipykernel_51172/2389888521.py:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  kohlenhydrate = nährwerte_div.find('span', text='Kohlenhydrate:')
/var/folders/m9/14p22wk9

In [143]:
# Erstelle DataFrame
data = {
    'Name': [essen.name for essen in essens_liste_detailled],
    'Foto': [essen.foto for essen in essens_liste_detailled],
    'Preis': [essen.preis for essen in essens_liste_detailled],
    'Vegan': [essen.vegan for essen in essens_liste_detailled],
    'CO2': [essen.co2 for essen in essens_liste_detailled],
    'Brennwert': [essen.brennwert for essen in essens_liste_detailled],
    'Fett': [essen.fett for essen in essens_liste_detailled],
    'Ges. Fett': [essen.ges_fett for essen in essens_liste_detailled],
    'Kohlenhydrate': [essen.kohlenhydrate for essen in essens_liste_detailled],
    'Zucker': [essen.zucker for essen in essens_liste_detailled],
    'Eiweiß': [essen.eiweiß for essen in essens_liste_detailled],
    'Salz': [essen.salz for essen in essens_liste_detailled]
}

df = pd.DataFrame(data)

In [144]:
def clean_nutrition_data(row):
    co2_portion_start = row['CO2'].find('CO2 pro Portion') + len('CO2 pro Portion')
    co2_portion_end = row['CO2'].find(' g', co2_portion_start)
    row['CO2 pro Portion'] = row['CO2'][co2_portion_start:co2_portion_end]

    co2_100g_start = row['CO2'].find('CO2 pro 100 g') + len('CO2 pro 100 g')
    co2_100g_end = row['CO2'].find(' g', co2_100g_start)
    row['CO2 pro 100 g'] = row['CO2'][co2_100g_start:co2_100g_end]

    brennwert_start = row['CO2'].find('Brennwert:') + len('Brennwert:')
    brennwert_end = row['CO2'].find(' kcal', brennwert_start)
    row['Brennwert'] = row['CO2'][brennwert_start:brennwert_end]

    fett_start = row['CO2'].find('Fett:') + len('Fett:')
    fett_end = row['CO2'].find(' g', fett_start)
    row['Fett'] = row['CO2'][fett_start:fett_end]

    ges_fett_start = row['CO2'].find('davon ges. FS:') + len('davon ges. FS:')
    ges_fett_end = row['CO2'].find(' g', ges_fett_start)
    row['Ges. Fett'] = row['CO2'][ges_fett_start:ges_fett_end]

    kohlenhydrate_start = row['CO2'].find('Kohlenhydrate:') + len('Kohlenhydrate:')
    kohlenhydrate_end = row['CO2'].find(' g', kohlenhydrate_start)
    row['Kohlenhydrate'] = row['CO2'][kohlenhydrate_start:kohlenhydrate_end]

    zucker_start = row['CO2'].find('davon Zucker:') + len('davon Zucker:')
    zucker_end = row['CO2'].find(' g', zucker_start)
    row['Zucker'] = row['CO2'][zucker_start:zucker_end]

    eiweiß_start = row['CO2'].find('Eiweiß:') + len('Eiweiß:')
    eiweiß_end = row['CO2'].find(' g', eiweiß_start)
    row['Eiweiß'] = row['CO2'][eiweiß_start:eiweiß_end]

    salz_start = row['CO2'].find('Salz:') + len('Salz:')
    salz_end = row['CO2'].find(' g', salz_start)
    row['Salz'] = row['CO2'][salz_start:salz_end]

    return row

In [145]:
# Annahme: Das DataFrame heißt df und die Spalte mit den Nährwertinformationen heißt 'CO2'
df = df.apply(clean_nutrition_data, axis=1)

In [146]:
# remove columns Foto and CO2
df = df.drop(columns=['Foto', 'CO2'])

In [147]:
df

,Name,Preis,Vegan,Brennwert,Fett,Ges. Fett,Kohlenhydrate,Zucker,Eiweiß,Salz,CO2 pro Portion,CO2 pro 100 g
0,Pilzcremesuppe,"€ 0,99 / 1,39",True,0.0 kj / 566.0,0.0,0.0,0.0,0.0,0.0,0.0,PortionBrennwert: 0.0 kj / 566.0 kcalFett: 0.0,o PortionBrennwert: 0.0 kj / 566.0 kcalFett: 0.0
1,"Vollkornreispfanne mit Zucchini, Paprika und A...","€ 2,99 / 5,49",True,2065.4 kj / 493.0,22.2,3.7,61.2,7.3,8.7,2.6,469.1,102.0
2,Lahmacun mit Salat,"€ 5,40 / 7,90",True,4236.3 kj / 999.9,70.9,7.2,73.2,11.6,20.1,7.2,385.8,78.7
3,Maultaschen mit Zwiebelsauce und Kartoffelsalat,"€ 4,90 / 7,40",True,2678.9 kj / 640.0,28.6,7.8,68.9,5.2,23.0,7.2,827.2,163.8
4,Gebackener Hirtenkäse auf bunterm Salatteller ...,"€ 4,80 / 7,30",True,3331.9 kj / 796.0,48.7,24.2,49.6,14.3,37.2,9.3,230.2,45.6
5,Burger Bar,"€ 1,10 / 1,60(je 100g)",True,3616.7 kj / 864.0,42.0,15.0,76.9,21.9,39.6,6.7,333.9,ion333.9
6,Wok im OG: Gemüsewok Curry mit Duftreis Hähnch...,"€ 1,10 / 1,60(je 100g)",True,3055.6 kj / 730.0,11.5,5.7,112.6,30.1,37.0,5.6,1681.3,ion1681.3
7,Wok im OG: Gemüsewok Curry mit Duftreis,"€ 1,10 / 1,60(je 100g)",True,2563.6 kj / 612.0,9.6,5.1,111.2,29.5,14.0,4.1,929.3,ion929.3
8,Flammkuchen Griechischer Art,"€ 4,95 / 7,45",True,3381.4 kj / 808.0,46.3,24.6,76.6,3.0,21.6,4.9,1291.3,430.4
9,Flammkuchen Elsässer Art,"€ 4,95 / 7,45",True,3222.2 kj / 770.0,39.0,19.2,81.1,3.0,23.7,4.7,1006.0,335.3


In [148]:
# save df to two new dfs called df_raw and df_clean
df_raw = df.copy()
df_clean = df.copy()

In [149]:
# clean the whole column Preis by extracting the number after the € sign
df_clean['Preis'] = df_clean['Preis'].str.extract(r'€\s*(\d+,\d+)')
df_clean['Preis'] = df_clean['Preis'].str.replace(',', '.').astype(float)

In [150]:
# clean column Brennwert by splitting each value at the character 'kj /' and taking the last part
df_clean['Brennwert'] = df_clean['Brennwert'].str.split(' kj /').str[-1]
df_clean['Brennwert'] = df_clean['Brennwert'].astype(float)

In [151]:
# remove the columns CO2 pro Portion and CO2 pro 100 g
df_clean = df_clean.drop(columns=['CO2 pro Portion', 'CO2 pro 100 g'])

In [152]:
# convert columns Fett, Ges. Fett, Kohlenhydrate, Zucker, Eiweiß and Salz to float
df_clean['Fett'] = df_clean['Fett'].astype(float)
df_clean['Ges. Fett'] = df_clean['Ges. Fett'].astype(float)
df_clean['Kohlenhydrate'] = df_clean['Kohlenhydrate'].astype(float)
df_clean['Zucker'] = df_clean['Zucker'].astype(float)
df_clean['Eiweiß'] = df_clean['Eiweiß'].astype(float)
df_clean['Salz'] = df_clean['Salz'].astype(float)

In [153]:
# create new columnns 'Brennwert pro Preis' and 'Eiweiß pro Preis' by dividing the columns 'Brennwert' and 'Eiweiß' by the column 'Preis'
df_clean['Brennwert pro Preis'] = df_clean['Brennwert'] / df_clean['Preis']
df_clean['Eiweiß pro Preis'] = df_clean['Eiweiß'] / df_clean['Preis']

In [154]:
# create a new df called df_recommend where there are only meals with a 'Preis' higher than 1.5
df_recommend = df_clean[df_clean['Preis'] > 1.5]

# show df_recommend sorted by 'Eiweiß pro Preis' and 'Brennwert pro Preis' in descending order
df_recommend = df_recommend.sort_values(by=['Eiweiß pro Preis', 'Brennwert pro Preis'], ascending=False)

,Name,Preis,Vegan,Brennwert,Fett,Ges. Fett,Kohlenhydrate,Zucker,Eiweiß,Salz,Brennwert pro Preis,Eiweiß pro Preis
4,Gebackener Hirtenkäse auf bunterm Salatteller ...,4.80,True,796.0,48.7,24.2,49.6,14.3,37.2,9.3,165.833333,7.750000
9,Flammkuchen Elsässer Art,4.95,True,770.0,39.0,19.2,81.1,3.0,23.7,4.7,155.555556,4.787879
3,Maultaschen mit Zwiebelsauce und Kartoffelsalat,4.90,True,640.0,28.6,7.8,68.9,5.2,23.0,7.2,130.612245,4.693878
11,Pizza Speziale,8.50,True,728.0,21.0,0.0,100.8,0.0,37.8,0.0,85.647059,4.447059
8,Flammkuchen Griechischer Art,4.95,True,808.0,46.3,24.6,76.6,3.0,21.6,4.9,163.232323,4.363636
10,Pizza mit Grillgemüse,8.50,True,793.0,18.5,10.3,114.8,28.7,36.9,4.9,93.294118,4.341176
2,Lahmacun mit Salat,5.40,True,999.9,70.9,7.2,73.2,11.6,20.1,7.2,185.166667,3.722222
1,"Vollkornreispfanne mit Zucchini, Paprika und A...",2.99,True,493.0,22.2,3.7,61.2,7.3,8.7,2.6,164.882943,2.909699


In [161]:
# get the weekday name of the date in str date
dateName = datetime.datetime.strptime(date, '%Y-%m-%d').strftime('%A')

print('Empfehlungen für '+dateName+', den '+date+': ')
print('-------------------------------------------')
df_recommend

Empfehlungen für Monday, den 2023-07-03: 
-------------------------------------------


,Name,Preis,Vegan,Brennwert,Fett,Ges. Fett,Kohlenhydrate,Zucker,Eiweiß,Salz,Brennwert pro Preis,Eiweiß pro Preis
4,Gebackener Hirtenkäse auf bunterm Salatteller ...,4.80,True,796.0,48.7,24.2,49.6,14.3,37.2,9.3,165.833333,7.750000
9,Flammkuchen Elsässer Art,4.95,True,770.0,39.0,19.2,81.1,3.0,23.7,4.7,155.555556,4.787879
3,Maultaschen mit Zwiebelsauce und Kartoffelsalat,4.90,True,640.0,28.6,7.8,68.9,5.2,23.0,7.2,130.612245,4.693878
11,Pizza Speziale,8.50,True,728.0,21.0,0.0,100.8,0.0,37.8,0.0,85.647059,4.447059
8,Flammkuchen Griechischer Art,4.95,True,808.0,46.3,24.6,76.6,3.0,21.6,4.9,163.232323,4.363636
10,Pizza mit Grillgemüse,8.50,True,793.0,18.5,10.3,114.8,28.7,36.9,4.9,93.294118,4.341176
2,Lahmacun mit Salat,5.40,True,999.9,70.9,7.2,73.2,11.6,20.1,7.2,185.166667,3.722222
1,"Vollkornreispfanne mit Zucchini, Paprika und A...",2.99,True,493.0,22.2,3.7,61.2,7.3,8.7,2.6,164.882943,2.909699
